In [1]:
import json, datetime, math
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.ensemble import GradientBoostingClassifier
import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.preprocessing import scale
import operator
from sklearn.metrics import calinski_harabaz_score
from sklearn.cluster import KMeans
import wordcloud

In [310]:
with open("data/orig_en_train.json") as f:
    data = json.load(f)
    # print("number of tweets", len(data))
with open("data/orig_en_test.json") as f:
    data_test = json.load(f)
    # print("number of tweets", len(data))

In [272]:
all_hs = defaultdict(int)
hs_likes = defaultdict(int)

for d in data:
    curr_hashtags = d['hashtags']
    curr_hashtags = curr_hashtags[1:-1].split(', ')
    curr_hashtags = [x for x in curr_hashtags if x != ""]
    for hs in curr_hashtags:
        if hs!="":
            all_hs[hs] += 1
    if int(d['like_count']) > 0:
        for hs in curr_hashtags:
            if hs!="":
                hs_likes[hs] += 1

sorted_hs = sorted(all_hs.items(), key=operator.itemgetter(1))
sorted_hsl = sorted(hs_likes.items(), key=operator.itemgetter(1))
    
quality_hs = []
quality_hsl = []
for hs in sorted_hs:
    if hs[1]>5:
        quality_hs.append(hs)
for hs in sorted_hsl:
    if hs[1]>5:
        quality_hsl.append(hs)

hs_index = defaultdict(int)
ind_hs = defaultdict(str)
for ind, hs in enumerate(quality_hs):
    hs_index[hs[0]] = ind
    ind_hs[ind] = hs[0]
    
all_user = set()
for d in data:
    all_user.add(d['userid'])
users = list(all_user)
user_ind = {}
ind_user = {}
user_hs = []
for ind, u in enumerate(users):
    user_hs.append([0]*len(quality_hs))
    user_ind[u] = ind
    ind_user[ind] = u
    
for d in data:
    curr_hashtags = d['hashtags']
    curr_hashtags = curr_hashtags[1:-1].split(', ')
    curr_hashtags = [x for x in curr_hashtags if x != ""]
    for hs in curr_hashtags:
        user_hs[user_ind[d['userid']]][hs_index[hs]] += 1

In [313]:
all_user_test = set()
for d in data_test:
    all_user_test.add(d['userid'])
users_test = list(all_user_test)

In [321]:
filtered_user_hs = []
user_filtered_ind = {}
i = 0
for ind, d in enumerate(user_hs):
    if d != [0]*len(quality_hs):
        filtered_user_hs.append(d)
        user_filtered_ind[ind_user[ind]] = i
        i += 1
filtered_user_hs = np.array(filtered_user_hs, dtype=np.float32)

In [274]:
user_hs_sums = []
for d in filtered_user_hs:
    user_hs_sums.append(sum(d))

In [275]:
for i in range(filtered_user_hs.shape[0]):
    filtered_user_hs[i] = np.divide(filtered_user_hs[i], user_hs_sums[i])

In [276]:
scaled_user_hs = scale(filtered_user_hs)

/home/santi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/home/santi/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [277]:
scaled_user_hs.shape

(1355, 604)

In [278]:
for i in range(2, 12):
    kmeans = KMeans(n_clusters=i, random_state=0).fit(scaled_user_hs)
    labels = kmeans.labels_
    centers = kmeans.cluster_centers_
    print(i, calinski_harabaz_score(scaled_user_hs, labels))

2 17.789531208414623
3 15.637888962963853
4 17.113145342996624
5 15.142467669124064
6 16.206088476725096
7 10.715249803852286
8 12.864664430700353
9 10.608776642685733
10 10.033058530229416
11 9.950221375328555


In [295]:
kmeans = KMeans(n_clusters=5, n_init=100).fit(scaled_user_hs)
labels = kmeans.labels_
centers = kmeans.cluster_centers_
print(kmeans.random_state)
sorted_center_hs = []
for d in centers:
    sorted_center_hs.append(np.argsort(d))

None


In [308]:
for d in sorted_center_hs[1][-20:]:
    print(ind_hs[d])

iHQ
GOPDebate
DeadHorse
KochFarms
IslamKills
quotes
America
BlackLivesMatter
Chernobyl2015
imho
ColumbianChemicals
Nukraine
USA
FukushimaAgain
BlackHistory
rap
Fukushima2015
quote
true
love


In [311]:
train_labels = labels

In [316]:
train_labels = train_labels + 1

In [317]:
np.unique(train_labels)

array([0, 1, 2, 3, 4], dtype=int32)

In [325]:
test_labels = []
for u in users_test:
    if u in user_filtered_ind:
        print(user_ind[u])
        test_labels.append(train_labels[user_filtered_ind[u]])
    else:
        test_labels.append(5)

0
1
2
3
5
7
8
9
10
11
12
13
14
15
16
17
18
19
21
22
23
24
25
26
27
28
29
30
31
35
37
38
40
41
43
45
47
48
49
51
53
54
56
58
59
60
61
66
67
68
69
71
72
73
76
77
79
80
81
82
84
86
87
88
91
92
93
94
95
96
97
98
99
100
102
101
103
104
105
106
107
108
109
110
111
112
114
115
116
117
118
119
121
122
123
124
125
126
132
134
136
139
140
141
144
145
146
148
149
150
151
152
154
156
157
159
160
163
164
165
167
168
169
170
172
174
173
175
177
178
179
181
180
183
184
186
187
188
189
190
193
194
195
196
199
201
203
204
206
207
209
208
211
213
214
215
216
217
218
219
220
221
223
224
225
226
228
229
231
232
233
236
237
239
241
242
243
245
244
248
251
254
255
256
257
259
260
261
262
263
264
265
266
268
269
270
271
273
275
277
278
279
280
281
283
284
286
288
289
290
291
292
293
294
295
296
298
299
300
303
304
305
310
311
312
314
315
318
317
316
321
322
326
327
328
329
330
331
332
335
336
339
338
340
341
343
344
346
345
347
349
350
351
353
354
355
356
357
358
360
361
362
363
366
365
367
368
370
371
372
3

In [324]:
import pickle

In [330]:
tweet_train_labels = []
tweet_test_labels = []
for d in data:
    if d['userid'] in user_filtered_ind: 
        tweet_train_labels.append(train_labels[user_filtered_ind[d['userid']]])
    else:
        tweet_train_labels.append(5)
for d in data_test:
    if d['userid'] in user_filtered_ind:
        tweet_test_labels.append(train_labels[user_filtered_ind[d['userid']]])
    else:
        tweet_test_labels.append(5)

In [332]:
np.array(tweet_train_labels).shape

(50000,)

In [336]:
with open("cluster_labels", "wb") as f:
    pickle.dump(tweet_train_labels, f)
    pickle.dump(tweet_test_labels, f)
    pickle.dump(kmeans, f)